In [3]:
import glob, re, os, json
from pyld import jsonld

In [1]:
ls ../


README.md*
Tables/
assets/
data_descriptors/
demo/
inputs/
obs4MIPs_frequency.json
obs4MIPs_grid_label.json
obs4MIPs_institution_id.json
obs4MIPs_license.json
obs4MIPs_nominal_resolution.json
obs4MIPs_product.json
obs4MIPs_realm.json
obs4MIPs_region.json
obs4MIPs_required_global_attributes.json
obs4MIPs_source_id.json
obs4MIPs_source_type.json
obs4MIPs_table_id.json
src/


In [ ]:
# source_type
data = json.load(open('../obs4MIPs_source_type.json'))['source_type']

for k,v in data.items():
    print(k)
    content = {
        "@context": "_context_",
        "description": v,
        "id": k.lower(),
        "name": k,
        "type": "source_type"
    }
    
    json.dump(content, open(f'../../WCRP-UNIVERSE/data_descriptors/source_type/{k.lower()}.json','w'), indent=4)

gridded_insitu gridded product based on measurements collected from in-situ instruments
reanalysis gridded product generated from a model reanalysis based on in-situ instruments and possibly satellite measurements
satellite_blended gridded product based on both in-situ instruments and satellite measurements
satellite_retrieval gridded product based on satellite measurements


In [13]:
# source
base = '../data_desciptors/source/'
os.system(f'mkdir -p {base}')

data = json.load(open('../obs4MIPs_source_id.json'))['source_id']

for k,v in data.items():
    print(v)
    v['id'] = k.lower()
    v['type'] = 'source'
    v['@context'] = '_context_'
    v['organisation'] = v['institution_id'].lower()
    del v['institution_id']
    v['source_type'] = v['source_type'].lower()
    v['description'] = v['source_description']
    del v['source_description']
    
    json.dump(v, open(f'{base}/{k.lower()}.json','w'), indent=4)

{'institution_id': 'NOAA-ESRL-PSD', 'region': ['global'], 'release_year': '2006', 'source_description': 'Monthly NOAA-CIRES 20th Century Reanalysis V2', 'source_id': '20CR-V2', 'source_label': '20CR', 'source_name': '20CR', 'source_type': 'reanalysis', 'source_variables': ['psl', 'ts'], 'source_version_number': 'V2'}
{'institution_id': 'NASA-JPL', 'region': ['global'], 'release_year': '2011', 'source_description': 'Atmospheric Infrared Sounder', 'source_label': 'AIRS', 'source_name': 'AIRS', 'source_type': 'satellite_retrieval', 'source_variables': ['hus', 'ta'], 'source_version_number': '1.0'}
{'institution_id': 'NASA-JPL', 'region': ['global'], 'release_year': '2011', 'source_description': 'Atmospheric Infrared Sounder', 'source_label': 'AIRS', 'source_name': 'AIRS', 'source_type': 'satellite_retrieval', 'source_variables': ['hus', 'ta'], 'source_version_number': '2.0'}
{'institution_id': 'RSS', 'region': ['global_ocean'], 'release_year': '2011', 'source_description': 'Sea Surface Te

In [ ]:
# tables and variables
tables = glob.glob('../Tables/*.json')[1:]
tables

print(tables)

skip = False
# tbl = tables[0]
for tbl in tables:
    for c in ['formula','coordinates','grids']: 
        if c in tbl:
            skip = True
    if skip:continue
    print(tbl)
    data = json.load(open(tbl,'r'))

    t = data['Header']
    t['id'] = t['table_id'].lower().split('_')[-1]
    t['type'] = 'mip_table'
    t['@context'] = '_context_'
    t['product'] = [t['product'].lower()]

    tpath = f'../../MIP-variables/data_descriptors/tables/{t["id"].lower()}.json'
    if os.path.exists(tpath):
        print(f'{tpath} exists',t['id'])
        data2 = json.load(open(tpath,'r'))
        data2['product'].extend(t['product'])
        json.dump(data2, open(tpath,'w'), indent=4)
    else:
        json.dump(t, open(tpath,'w'), indent=4)

    for vi,v in data['variable_entry'].items():
        v['id'] = vi.lower()
        v['type'] = 'mip-variable'
        v['@context'] = '_context_'
        v["mip_tables"]: [
            {
                "@id": t['id'],
                "mip-era": "obs4mips"
            }
        ]
        v['themes'] = ['TBC']
        v["modeling_realm"]= [v["modeling_realm"].lower()]
        
        vpath = f'../../MIP-variables/data_descriptors/variables/{t["id"]}.{v["id"]}.json'
        if os.path.exists(vpath):
            print(f'{vpath} exists',v['id'])
            data2 = json.load(open(vpath,'r'))
            if 'mip_tables' not in data2:
                data2['mip_tables']= []
            data2['mip_tables'].extend(t['id'])
            json.dump(data2, open(vpath,'w'), indent=4)
        else:
            json.dump(v, open(vpath,'w'), indent=4)

['../Tables/obs4MIPs_SImon.json', '../Tables/obs4MIPs_formula_terms.json', '../Tables/obs4MIPs_Oday.json', '../Tables/obs4MIPs_A6hr.json', '../Tables/obs4MIPs_coordinate-ERA5levs.json', '../Tables/obs4MIPs_Amon.json', '../Tables/obs4MIPs_A1hr.json', '../Tables/obs4MIPs_Lmon.json', '../Tables/obs4MIPs_fx.json', '../Tables/obs4MIPs_Aday.json', '../Tables/obs4MIPs_A3hr.json', '../Tables/obs4MIPs_SIday.json', '../Tables/obs4MIPs_grids.json', '../Tables/obs4MIPs_coordinate.json', '../Tables/obs4MIPs_Omon.json']
../Tables/obs4MIPs_SImon.json
../../MIP-variables/data_descriptors/tables/simon.json exists simon
../../MIP-variables/data_descriptors/variables/simon.sfdsi.json exists sfdsi
../../MIP-variables/data_descriptors/variables/simon.siage.json exists siage
../../MIP-variables/data_descriptors/variables/simon.siareaacrossline.json exists siareaacrossline
../../MIP-variables/data_descriptors/variables/simon.siarean.json exists siarean
../../MIP-variables/data_descriptors/variables/simon.sia

KeyError: 'Header'

In [49]:
tables

['../Tables/obs4MIPs_SImon.json',
 '../Tables/obs4MIPs_formula_terms.json',
 '../Tables/obs4MIPs_Oday.json',
 '../Tables/obs4MIPs_A6hr.json',
 '../Tables/obs4MIPs_coordinate-ERA5levs.json',
 '../Tables/obs4MIPs_Amon.json',
 '../Tables/obs4MIPs_A1hr.json',
 '../Tables/obs4MIPs_Lmon.json',
 '../Tables/obs4MIPs_fx.json',
 '../Tables/obs4MIPs_Aday.json',
 '../Tables/obs4MIPs_A3hr.json',
 '../Tables/obs4MIPs_SIday.json',
 '../Tables/obs4MIPs_grids.json',
 '../Tables/obs4MIPs_coordinate.json',
 '../Tables/obs4MIPs_Omon.json']

In [ ]:
  "cell_measures":"area: areacella",
            "cell_methods":"area: time: mean",
            "comment":"Total cloud area fraction (reported as a percentage) for the whole atmospheric column, as seen from the surface or the top of the atmosphere. Includes both large-scale and convective cloud.",
            "dimensions":"longitude latitude time",
            "frequency":"3hr",
            "long_name":"Total Cloud Cover Percentage",
            "modeling_realm":"atmos",
            "ok_max_mean_abs":"",
            "ok_min_mean_abs":"",
            "out_name":"clt",
            "positive":"",
            "standard_name":"cloud_area_fraction",
            "type":"real",
            "units":"%",
            "valid_max":"",
            "valid_min":""
            
                "@context": "_context_",
    "id": "acmonz.oxloss",
    "type": "mip-variable",
    "mip_tables": [
        {
            "@id": "acmonz",
            "mip-era": "cmip6plus"
        }
    ],
    "themes": [
        "TBC"
    ],
    "cell_measures": "",
    "cell_methods": "longitude: mean time: mean",
    "comment": "total chemical loss rate for o+o1d+o3",
    "dimensions": [
        "latitude",
        "plev39",
        "time"
    ],
    "frequency": "mon",
    "long_name": "Total Odd Oxygen (Ox) Loss Rate",
    "modeling_realm": [
        "atmosChem"
    ],
    "ok_max_mean_abs": "",
    "ok_min_mean_abs": "",
    "out_name": "oxloss",
    "positive": "",
    "standard_name": "tendency_of_mole_concentration_of_ox_in_air_due_to_chemical_destruction",
    "units": "mol m-3 s-1",
    "valid_max": "",
    "valid_min": "",
    "dtype": "real"